**Simulation of the EBS at Quezon City**

In [1]:
from beyond.io.tle import Tle
from beyond.dates import Date, timedelta
from beyond.frames import create_station
import requests
from datetime import datetime, timedelta
import ephem

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
import sys

In [2]:
celestrak_active =  "https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle"

data = requests.get(celestrak_active)
data = data.content.decode("utf-8")
rows = data.splitlines()

tles = {}
first = None
second = None
last = None
for row in rows:
    if not first:
        first = row
    elif not second:
        second = row
    else:
        last = row
        name = (first.split("[")[0]).strip()
        tles[name] = """%s
%s
%s""" % (first, second, last)
        first = None
        second= None

In [2]:
# March 16, 9:36 PM

tle_iss = Tle("""ISS (ZARYA)
1 25544U 98067A   24076.34553448  .00011668  00000+0  21954-3 0  9990
2 25544  51.6403  55.5567 0004480 331.2605 172.8778 15.49007292444146
""").orbit()

tle_abs = Tle("""ABS-6
1 25924U 99053A   24075.84552637 -.00000131  00000+0  00000+0 0  9990
2 25924   0.0201 253.7679 0002690  87.8524 295.6836  1.00271566 89584
""").orbit()

**Functions**

In [3]:
def orbital_passes(tle, station, hours, interval, date_time, fov_horizontal, fov_vertical, fov_center=(0, 0)):
    date_time_info = Date(date_time.year, date_time.month, date_time.day, date_time.hour, date_time.minute, date_time.second)

    passes_data = []
    passes_in_fov = []
    pass_info = {}

    print("    Time      Azim    Elev    Distance   Radial Velocity")
    print("=========================================================")

    # Model propagation, rf conversion, azim and elev calculation
    for orb in station.visibility(tle, start=date_time_info, stop=timedelta(hours=hours), step=timedelta(seconds=interval), events=True):
        elev = np.degrees(orb.phi)
        azim = np.degrees(-orb.theta) % 360

        event_info = orb.event.info if orb.event is not None else ""
        if "AOS" in event_info:
            pass_info = {"AOS_time": orb.date, "AOS_azim": azim, "AOS_elev": 90 - elev}
            passes_data.append(pass_info)

        if "MAX" in event_info:
            pass_info["MAX_time"] = orb.date
            pass_info["MAX_elev"] = elev
            pass_info["MAX_azim"] = azim

        if "LOS" in event_info and len(passes_data) > 0:
            passes_data[-1]["LOS_time"] = orb.date
            passes_data[-1]["LOS_azim"] = azim
            passes_data[-1]["LOS_elev"] = 90 - elev
        
        r = orb.r / 1000.
        print("{event:7} {orb.date:%H:%M:%S} {azim:7.2f} {elev:7.2f} {r:10.2f} {orb.r_dot:10.2f}".format(
            orb=orb, r=r, azim=azim, elev=elev, event=orb.event.info if orb.event is not None else ""
        ))

        if (fov_center[0] - fov_horizontal / 2) <= azim <= (fov_center[0] + fov_horizontal / 2) and (fov_center[1] - fov_vertical / 2) <= elev <= (fov_center[1] + fov_vertical / 2):
            passes_in_fov.append((orb.date, azim, elev))

    # Passes data        
    df_passes = pd.DataFrame(passes_data)

# ---------------------------------- for additional info --------------------------------------
    # Passes summary
    pass_number = range(1, len(df_passes) + 1)
    df_passes_info = pd.DataFrame({
        'Pass_number': pass_number,
        'AOS_time': df_passes['AOS_time'].values,  
        'LOS_time': df_passes['LOS_time'].values,
        'Duration': df_passes['LOS_time'] - df_passes['AOS_time'],  
        'MAX_azim': df_passes['MAX_azim'],
        'MAX_elev': df_passes['MAX_elev']
    })
    print(df_passes_info)

    # Polar plot
    plt.figure(figsize=(10, 8))
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')

    pass_colors = sns.color_palette("husl", len(df_passes))
    legend_handles = []
    for i, (_, pass_info) in enumerate(df_passes.iterrows()):
    
        plt.plot(np.radians(pass_info["AOS_azim"]), pass_info["AOS_elev"], 'o', color=pass_colors[i], markersize=8, markeredgewidth=2, markerfacecolor='none')
        plt.plot(np.radians(pass_info["LOS_azim"]), pass_info["LOS_elev"], 'o', color=pass_colors[i], markersize=12)
        plt.plot(np.radians(pass_info["MAX_azim"]), 90 - pass_info["MAX_elev"], '*', color=pass_colors[i], markersize=6)

        points_azim = []
        points_elev = []
        for orb in station.visibility(tle, start=pass_info["AOS_time"], stop=pass_info["LOS_time"], step=timedelta(seconds=30), events=False):
            elev = np.degrees(orb.phi)
            azim = np.degrees(-orb.theta) % 360
            points_azim.append(azim)
            points_elev.append(90 - elev)

        plt.plot(np.radians(points_azim), points_elev, color=pass_colors[i])
        legend_handles.append(plt.Line2D([0], [0], color=pass_colors[i], lw=2, label=f"Pass {i+1}"))

    plt.legend(handles=legend_handles)

    # Event-based sensor field of view
    fov_left = np.radians(fov_center[0] - fov_horizontal / 2)
    fov_bottom = 90 - (fov_center[1] + fov_vertical / 2)
    fov_width = np.radians(fov_horizontal)
    fov_height = fov_vertical
    fov_rect_patch = Rectangle((fov_left, fov_bottom), fov_width, fov_height, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(fov_rect_patch)

    ax.set_aspect('equal', adjustable='box')
    ax.set_yticks(range(0, 90, 20))
    ax.set_yticklabels(map(str, range(90, 0, -20)))
    ax.set_rmax(90)

    label_distance = 110
    for angle, label in zip([0, 90, 180, 270], ['N', 'E', 'S', 'W']):
        ax.text(np.radians(angle), label_distance, label, ha='center', va='center')

    plt.show()

    plot_passes_within_fov_ebs(passes_in_fov, fov_horizontal, fov_vertical, fov_center, pass_colors, df_passes)
    
    return df_passes_info


In [4]:
def plot_passes_within_fov_ebs(passes_in_fov, fov_horizontal, fov_vertical, fov_center, pass_colors, df_passes):
    if passes_in_fov:
        plt.figure(figsize=(12, 8))
        
        prev_pass_num = None
        prev_azim_px = None
        prev_elev_px = None
        updated_passes_in_fov = []

        # FOV validation with the EBS parameters
        for timestamp, azim, elev in passes_in_fov:
            if (fov_center[0] - fov_horizontal / 2) <= azim <= (fov_center[0] + fov_horizontal / 2) and \
                    (fov_center[1] - fov_vertical / 2) <= elev <= (fov_center[1] + fov_vertical / 2):
                pass_num, color = find_pass_info(df_passes, azim, elev, pass_colors)

                # Degrees to pixels conversion
                azim_arcsec = azim * 3600
                elev_arcsec = elev * 3600
                azim_px = (azim_arcsec - (fov_center[0] - fov_horizontal / 2) * 3600) * (240 / (fov_horizontal * 3600))
                elev_px = (elev_arcsec - (fov_center[1] - fov_vertical / 2) * 3600) * (180 / (fov_vertical * 3600))
                updated_passes_in_fov.append((timestamp, azim, elev, azim_px, elev_px))
                plt.plot(azim_px, elev_px, linestyle='-', color=color)

                if prev_pass_num is not None and prev_pass_num == pass_num:
                    plt.plot([prev_azim_px, azim_px], [prev_elev_px, elev_px], color=color)
                prev_pass_num = pass_num
                prev_azim_px = azim_px
                prev_elev_px = elev_px

        passes_in_fov_df = pd.DataFrame(updated_passes_in_fov, columns=['timestamp', 'azim', 'elev', 'x', 'y'])
        print(passes_in_fov_df) 

        first_point_time = passes_in_fov_df.iloc[0]['timestamp']
        last_point_time = passes_in_fov_df.iloc[-1]['timestamp']
        first_point_azim, first_point_elev = passes_in_fov_df.iloc[0][['x', 'y']]
        last_point_azim, last_point_elev = passes_in_fov_df.iloc[-1][['x', 'y']]
        sns.set_style("whitegrid")
        plt.plot(first_point_azim, first_point_elev, 'o', color = color, markerfacecolor = 'none') 
        plt.plot(last_point_azim, last_point_elev, 'o', color = color) 
        label_offset_x = 5
        label_offset_y = 10
        plt.text(first_point_azim, first_point_elev - label_offset_y, f'Start: {first_point_time}', fontsize=10, ha='right', va='bottom')
        plt.text(last_point_azim - label_offset_x, last_point_elev - label_offset_x, f'End: {last_point_time}', fontsize=10, ha='left', va='bottom')
        plt.xlim(0, 240)
        plt.ylim(0, 180)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True)
        plt.show()
    else:
        print('No passes within the specified field of view.')

def find_pass_info(df_passes, azim, elev, pass_colors):
    min_distance = float('inf')
    
    for i, pass_info in df_passes.iterrows():
        distance = abs(pass_info["AOS_azim"] - azim) + abs(pass_info["AOS_elev"] - (90 - elev))
        if distance < min_distance:
            min_distance = distance
            closest_pass = i
    
    return closest_pass, pass_colors[closest_pass]


**ISS SIMULATION AT QUEZON CITY ON MARCH 16, 2024**

In [ ]:
orbital_passes(tle_iss, create_station('Quezon City', (14.65,121.05,0)), 24,5, datetime(2024, 3, 16, 0, 0, 0), 0,0, fov_center=(0, 0))

*With no star tracking*

I chose the pass that is most visible from the station (pass 4) and used its max elev and azim as the center of the fov. I also set the timestep to be 0.0006s for more data points

In [ ]:
orbital_passes(tle_iss, create_station('Quezon City', (14.65,121.05,0)), 0.25,0.0006, datetime(2024, 3, 16, 19, 7, 0), 0.124,0.093, fov_center=(233.681366, 60.675961))

**ABS-6 SIMULATION AT QUEZON CITY ON MARCH 16, 2024**

*With no star tracking*

In [4]:
def orbital_passes(tle, station, hours, interval, specific_date, fov_horizontal, fov_vertical, fov_center=(0, 0)):
    passes_in_fov = []

    print("    Time      Azim    Elev    Distance   Radial Velocity")
    print("=========================================================")

    for orb in station.visibility(tle, start=specific_date, stop=timedelta(hours=hours), step=timedelta(seconds=interval), events=True):
        elev = np.degrees(orb.phi)
        azim = np.degrees(-orb.theta) % 360

        r = orb.r / 1000.
        print("{event:7} {orb.date:%H:%M:%S} {azim:7.2f} {elev:7.2f} {r:10.2f} {orb.r_dot:10.2f}".format(
            orb=orb, r=r, azim=azim, elev=elev, event=orb.event.info if orb.event is not None else ""
        ))

        # Check if pass is within FOV
        if (fov_center[0] - fov_horizontal / 2) <= azim <= (fov_center[0] + fov_horizontal / 2) and (fov_center[1] - fov_vertical / 2) <= elev <= (fov_center[1] + fov_vertical / 2):
            passes_in_fov.append((orb.date, azim, elev)) 
            
    # Pass data
    df_passes = pd.DataFrame(passes_in_fov, columns=['timestamp', 'azim', 'elev'])
    sns.set(style="whitegrid", palette="muted")
    plt.figure(figsize=(10, 8))
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.scatter(np.radians(df_passes['azim']), df_passes['elev'], s=20, marker='o', label='Pass') 

    # Rectangle patch
    fov_left = np.radians(fov_center[0] - fov_horizontal / 2)
    fov_bottom = 90 - (fov_center[1] + fov_vertical / 2)
    fov_width = np.radians(fov_horizontal)
    fov_height = fov_vertical
    fov_rect_patch = Rectangle((fov_left, fov_bottom), fov_width, fov_height, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(fov_rect_patch)
    ax.set_aspect('equal', adjustable='box')
    
    ax.set_yticks(range(0, 90, 20))
    ax.set_yticklabels(map(str, range(90, 0, -20)))
    ax.set_rmax(90)
    label_distance = 110
    for angle, label in zip([0, 90, 180, 270], ['N', 'E', 'S', 'W']):
        ax.text(np.radians(angle), label_distance, label, ha='center', va='center') 
    plt.show()

    plot_passes_within_fov_ebs(passes_in_fov, fov_horizontal, fov_vertical, fov_center)

    return df_passes


In [11]:
def plot_passes_within_fov_ebs(passes_in_fov, fov_horizontal, fov_vertical, fov_center):
    if passes_in_fov:
        plt.figure(figsize=(12, 8))

        prev_azim_px = None
        prev_elev_px = None
        updated_passes_in_fov = []

        sns.set_palette("muted")
        for timestamp, azim, elev in passes_in_fov:
            if (fov_center[0] - fov_horizontal / 2) <= azim <= (fov_center[0] + fov_horizontal / 2) and \
                    (fov_center[1] - fov_vertical / 2) <= elev <= (fov_center[1] + fov_vertical / 2):
                
                # Convert degrees to arcseconds
                azim_arcsec = azim * 3600
                elev_arcsec = elev * 3600
                azim_px = (azim_arcsec - (fov_center[0] - fov_horizontal / 2) * 3600) * (240 / (fov_horizontal * 3600))
                elev_px = (elev_arcsec - (fov_center[1] - fov_vertical / 2) * 3600) * (180 / (fov_vertical * 3600))
                updated_passes_in_fov.append((timestamp, azim, elev, azim_px, elev_px))

                plt.plot(azim_px, elev_px, linestyle='-', color='blue') 

                # Connect the points with lines only if they belong to the same pass
                if prev_azim_px is not None and prev_elev_px is not None:
                    plt.plot([prev_azim_px, azim_px], [prev_elev_px, elev_px], color=sns.color_palette("muted")[0])  # Plotting in blue

                prev_azim_px = azim_px
                prev_elev_px = elev_px

        passes_in_fov_df = pd.DataFrame(updated_passes_in_fov, columns=['timestamp', 'azim', 'elev', 'x', 'y'])
        print(passes_in_fov_df)  
        first_point_time = passes_in_fov_df.iloc[0]['timestamp']
        last_point_time = passes_in_fov_df.iloc[-1]['timestamp']
        first_point_azim, first_point_elev = passes_in_fov_df.iloc[0][['x', 'y']]
        last_point_azim, last_point_elev = passes_in_fov_df.iloc[-1][['x', 'y']]
        label_offset = 5 

        plt.scatter(passes_in_fov_df.iloc[0]['x'], passes_in_fov_df.iloc[0]['y'], color='red', label='Start') 
        plt.scatter(passes_in_fov_df.iloc[-1]['x'], passes_in_fov_df.iloc[-1]['y'], color='blue', label='End') 
        plt.text(first_point_azim -label_offset, first_point_elev + label_offset, f'Start: {first_point_time}', fontsize=10, ha='left', va='top', color='red')
        plt.text(last_point_azim - label_offset, last_point_elev - label_offset, f'End: {last_point_time}', fontsize=10, ha='left', va='top', color='blue')
        plt.xlim(0, 240)
        plt.ylim(0, 180)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True)
        plt.show()
    else:
        print('No passes within the specified field of view.')


In [ ]:
pass_info_df = orbital_passes(tle_abs, create_station('Quezon City', (14.65,121.05,0)), 24, 10, Date(2024,3,16, scale="UTC"),0.124,0.093, fov_center=(107.910898,43.414817))

*With star tracking*

In [15]:
def plot_passes_within_fov_ebs(passes_in_fov, fov_horizontal, fov_vertical, fov_center):
    westward_speed = 0.0042  # Westward speed in degrees per second
    if passes_in_fov:
        plt.figure(figsize=(12, 8))

        prev_azim_px = None
        prev_elev_px = None
        updated_passes_in_fov = []

        sns.set_palette("muted")
        start_time = passes_in_fov[0][0]  # Get the start time of the first pass
        point_at_azim_zero = None  # To store the point with azimuth pixel = 0
        for timestamp, azim, elev in passes_in_fov:
            if (fov_center[0] - fov_horizontal / 2) <= azim <= (fov_center[0] + fov_horizontal / 2) and \
                    (fov_center[1] - fov_vertical / 2) <= elev <= (fov_center[1] + fov_vertical / 2):
                
                # Convert degrees to arcseconds
                azim_arcsec = azim * 3600
                elev_arcsec = elev * 3600
                azim_px = (azim_arcsec - (fov_center[0] - fov_horizontal / 2) * 3600) * (240 / (fov_horizontal * 3600))
                elev_px = (elev_arcsec - (fov_center[1] - fov_vertical / 2) * 3600) * (180 / (fov_vertical * 3600))
                
                # Calculate time difference in seconds
                time_difference = (timestamp - start_time).total_seconds()
                
                # Adjust for the westward movement
                azim_px -= westward_speed * time_difference * (240 / (fov_horizontal * 3600))

                updated_passes_in_fov.append((timestamp, azim, elev, azim_px, elev_px))
                
                # Check if azimuth pixel is closest to 0
                if azim_px == 0 or (point_at_azim_zero is None and azim_px < 0):
                    point_at_azim_zero = (timestamp, azim, elev, azim_px, elev_px)

                plt.plot(azim_px, elev_px, linestyle='-', color='blue') 

                # Connect the points with lines only if they belong to the same pass
                if prev_azim_px is not None and prev_elev_px is not None:
                    plt.plot([prev_azim_px, azim_px], [prev_elev_px, elev_px], color=sns.color_palette("muted")[0])  # Plotting in blue

                prev_azim_px = azim_px
                prev_elev_px = elev_px

        passes_in_fov_df = pd.DataFrame(updated_passes_in_fov, columns=['timestamp', 'azim', 'elev', 'x', 'y'])
        print(passes_in_fov_df)  
        first_point_time = passes_in_fov_df.iloc[0]['timestamp']
        last_point_time = passes_in_fov_df.iloc[-1]['timestamp']
        first_point_azim, first_point_elev = passes_in_fov_df.iloc[0][['x', 'y']]
        label_offset = 5

        plt.scatter(passes_in_fov_df.iloc[0]['x'], passes_in_fov_df.iloc[0]['y'], color='red', label='Start') 
        plt.text(first_point_azim + label_offset, first_point_elev + label_offset, f'Start: {first_point_time}', fontsize=10, ha='left', va='top', color='red')
        
        # Set the last point with timestamp label to the point with azimuth pixel = 0, if found
        if point_at_azim_zero is not None:
            timestamp, azim, elev, azim_px, elev_px = point_at_azim_zero
            plt.scatter(azim_px, elev_px, color='blue')  # Plotting the last point at azimuth pixel = 0
            plt.text(azim_px + (10. * label_offset), elev_px + label_offset, f'End: {timestamp}', fontsize=10, ha='right', va='bottom', color='blue')

        plt.xlim(0, 240)
        plt.ylim(0, 180)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid(True)
        plt.show()

        if len(updated_passes_in_fov) > 0:
            last_point_azim, last_point_elev = updated_passes_in_fov[-1][3:5]

        print("Last Point (azim, elev):", last_point_azim, last_point_elev)
    else:
        print('No passes within the specified field of view.')


In [ ]:
pass_info_df = orbital_passes(tle_abs, create_station('Quezon City', (14.65,121.05,0)), 24, 10, Date(2024,3,16, scale="UTC"),0.124,0.093, fov_center=(107.910898,43.414817))

**ABS-6 SIMULATION AT ADELAIDE WITH STAR TRACKING** \
June 4, 2017

In [ ]:
def orbital_passes_elev_azim(elevation, azimuth):
    azims, elevs = [], []

    print("Azim    Elev")
    print("==============")

    azim = azimuth
    elev = elevation

    azims.append(azim)

    elevs.append(90 - elev)

    print("{azim:7.2f} {elev:7.2f}".format(azim=azim, elev=elev))

    plt.figure(figsize=(10, 8))
    ax = plt.subplot(111, projection='polar')
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    plt.plot(np.radians(azims), elevs, '.')

    ax.set_yticks(range(0, 90, 20))
    ax.set_yticklabels(map(str, range(90, 0, -20)))
    ax.set_rmax(90)

    label_distance = 120 
    for angle, label in zip([0, 90, 180, 270], ['N', 'E', 'S', 'W']):
        ax.text(np.radians(angle), label_distance, label, ha='center', va='center')

    ax.set_yticks(range(0, 90, 20))
    ax.set_yticklabels(map(str, range(90, 0, -20)))
    ax.set_rmax(90)

    plt.show()

azimuth = 32.8
elev = 43.5
orbital_passes_elev_azim(elev, azimuth)